In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
import math
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inlineb
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
mid = pd.read_csv("MID pre-inj.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    1.247000e+03    1.247000e+03     1.247000e+03     1.247000e+03   
mean     4.684629e-11    5.319021e-13     3.761891e-13     7.634797e-13   
std      1.038668e-11    2.880055e-13     5.705851e-14     2.929835e-13   
min      3.559680e-11    6.100000e-15     1.769000e-13     2.872000e-13   
25%      4.040770e-11    9.015000e-14     3.485000e-13     7.066000e-13   
50%      4.576630e-11    6.968000e-13     3.822000e-13     7.588000e-13   
75%      5.199670e-11    7.240500e-13     4.050500e-13     8.033000e-13   
max      2.121520e-10    1.036000e-12     1.211300e-12     1.066500e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     1.247000e+03     1.247000e+03     1.247000e+03     1.245000e+03   
mean      1.832510e-12     2.797094e-12     5.040104e-14     4.605213e-13   
std       4.499432e-13     4.316036e-13     7.112353e-14     5.082496e-13   
min       1.1113

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/28 17:11:00.339,2022/11/28 17:11:03.590,2022/11/28 17:11:03.840,2022/11/28 17:11:04.090,2022/11/28 17:11:04.340,2022/11/28 17:11:04.590,2022/11/28 17:11:04.840,2022/11/28 17:11:05.091,2022/11/28 17:11:05.341,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2022/11/28 17:11:06.163,2022/11/28 17:11:09.414,2022/11/28 17:11:09.664,2022/11/28 17:11:09.914,2022/11/28 17:11:10.164,2022/11/28 17:11:10.414,2022/11/28 17:11:10.664,2022/11/28 17:11:10.915,2022/11/28 17:11:11.165,5.824,5.824,5.824,5.824,5.824,5.824,5.824,5.824,5.824
4,2022/11/28 17:11:11.987,2022/11/28 17:11:15.238,2022/11/28 17:11:15.488,2022/11/28 17:11:15.738,2022/11/28 17:11:15.989,2022/11/28 17:11:16.239,2022/11/28 17:11:16.489,2022/11/28 17:11:16.739,2022/11/28 17:11:16.989,11.648,11.648,11.648,11.648,11.649,11.649,11.649,11.648,11.648
5,2022/11/28 17:11:17.812,2022/11/28 17:11:21.063,2022/11/28 17:11:21.313,2022/11/28 17:11:21.563,2022/11/28 17:11:21.814,2022/11/28 17:11:22.064,2022/11/28 17:11:22.314,2022/11/28 17:11:22.564,2022/11/28 17:11:22.814,17.473,17.473,17.473,17.473,17.474,17.474,17.474,17.473,17.473
6,2022/11/28 17:11:23.637,2022/11/28 17:11:26.888,2022/11/28 17:11:27.138,2022/11/28 17:11:27.388,2022/11/28 17:11:27.639,2022/11/28 17:11:27.889,2022/11/28 17:11:28.139,2022/11/28 17:11:28.389,2022/11/28 17:11:28.639,23.298,23.298,23.298,23.298,23.299,23.299,23.299,23.298,23.298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,2022/11/28 17:59:35.904,2022/11/28 17:59:36.154,2022/11/28 17:59:36.404,2022/11/28 17:59:36.654,2022/11/28 17:59:36.904,2022/11/28 17:59:37.155,2022/11/28 17:59:37.405,2022/11/28 17:59:37.655,2022/11/28 17:59:37.905,2915.565,2912.564,2912.564,2912.564,2912.564,2912.565,2912.565,2912.564,2912.564
1245,2022/11/28 17:59:38.226,2022/11/28 17:59:38.476,2022/11/28 17:59:38.727,2022/11/28 17:59:38.977,2022/11/28 17:59:39.227,2022/11/28 17:59:39.477,2022/11/28 17:59:39.727,2022/11/28 17:59:40.228,NaN,2917.887,2914.886,2914.887,2914.887,2914.887,2914.887,2914.887,2915.137,NaN
1246,2022/11/28 17:59:40.549,2022/11/28 17:59:40.799,2022/11/28 17:59:41.049,2022/11/28 17:59:41.299,2022/11/28 17:59:41.549,2022/11/28 17:59:41.800,2022/11/28 17:59:42.050,2022/11/28 17:59:42.550,NaN,2920.210,2917.209,2917.209,2917.209,2917.209,2917.210,2917.210,2917.459,NaN
1247,2022/11/28 17:59:42.871,2022/11/28 17:59:43.121,2022/11/28 17:59:43.372,2022/11/28 17:59:43.622,2022/11/28 17:59:43.872,2022/11/28 17:59:44.122,2022/11/28 17:59:44.372,2022/11/28 17:59:44.873,NaN,2922.532,2919.531,2919.532,2919.532,2919.532,2919.532,2919.532,2919.782,NaN


In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,28/11/2022 16:32:45,8.0,0.58611,1.210000e-09,1.320000e-09,4.996000e-09,3.540000e-07,0.65517,175.0,-5.163,4269.231,NaN,NaN
2,NaN,28/11/2022 16:32:52,15.0,0.58390,1.210000e-09,1.320000e-09,4.996000e-09,3.540000e-07,0.65514,173.0,-5.050,4269.231,NaN,NaN
3,NaN,28/11/2022 16:32:59,22.0,0.58274,1.210000e-09,1.320000e-09,4.996000e-09,3.540000e-07,0.65516,173.0,-4.938,4268.894,NaN,NaN
4,NaN,28/11/2022 16:33:06,29.0,0.58239,1.200000e-09,1.320000e-09,4.996000e-09,3.540000e-07,0.65517,173.0,-4.938,4269.006,NaN,NaN
5,NaN,28/11/2022 16:33:13,36.0,0.58533,1.210000e-09,1.310000e-09,4.996000e-09,3.540000e-07,0.65515,172.0,-5.275,4268.670,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,4341.000000,4341.000000,4341.000000,4341.000000,4341.000000,4341.000000,4341.0,4.341000e+03,4341.0,4341.000000,4341.000000,4341.000000,4341.000000,4341.000000,4341.000000,4341.000000,4341.000000,4.341000e+03,4.341000e+03
mean,2583.251544,111.399714,54.532182,528.113098,526.853273,525.725777,0.0,2.000000e-02,0.0,-0.061605,0.478516,15.469376,15.415588,15.301603,-0.017593,-0.026843,-0.019991,7.129307e-08,5.016085e-07
std,1500.676592,207.604019,49.781056,497.432837,497.522652,497.706431,0.0,3.122862e-17,0.0,0.106352,0.753461,14.835321,14.882397,14.919664,0.002206,0.001190,0.001055,1.001140e-07,7.382854e-07
min,1.185000,0.000000,0.080000,0.000000,0.000000,0.000000,0.0,2.000000e-02,0.0,-0.782400,-3.641600,-23.742600,-27.902400,-29.544500,-0.022900,-0.028700,-0.024000,0.000000e+00,8.000000e-10
25%,1280.431000,0.040000,0.080000,0.080000,0.100000,0.000000,0.0,2.000000e-02,0.0,-0.202400,0.008200,0.007800,0.008800,0.005800,-0.018400,-0.027400,-0.020400,2.400000e-09,8.200000e-09
50%,2566.263000,0.080000,100.080000,999.720000,999.760000,999.760000,0.0,2.000000e-02,0.0,-0.002400,0.027000,28.065400,28.051500,28.008700,-0.017600,-0.027000,-0.019900,1.670000e-08,2.730000e-08
75%,3886.783000,0.140000,100.080000,999.900000,1000.000000,1000.020000,0.0,2.000000e-02,0.0,0.000200,1.441800,29.760400,29.715100,29.715800,-0.016200,-0.026100,-0.019400,2.032000e-07,1.446600e-06
max,5193.555000,499.900000,101.940000,1000.060000,1000.080000,1000.240000,0.0,2.000000e-02,0.0,0.549200,3.512000,30.601100,30.624700,30.612700,-0.014200,-0.023700,-0.017800,7.824000e-07,3.641600e-06


### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="emission on again"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 11, 28, 17,20,20)
y=pd.Timestamp(2022, 11, 28, 17,27,22)

#then find the time value closest to it in the MID scan file 
print("\n","Result 3:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

#remove values under noise level from MID spectrum
mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][233:414]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:37]/h2_sens
base_h2.describe()

Result 1:             Date          Time  Elapsed time  VMon1  VMon2   VMon3   VMon4  \
2388  28/11/2022  17:20:19.199      2844.881   0.08  100.1  999.78  999.88   
2389  28/11/2022  17:20:20.386      2846.068   0.08  100.1  999.78  999.88   
2390  28/11/2022  17:20:21.571      2847.253   0.10  100.1  999.78  999.88   
2391  28/11/2022  17:20:22.800      2848.482   0.10  100.1  999.78  999.88   

       VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
2388  999.88    0.0   0.02  ...  1.6524  30.2496  30.2587  30.2626 -0.0184   
2389  999.88    0.0   0.02  ...  1.6548  30.2472  30.2537  30.2593 -0.0184   
2390  999.88    0.0   0.02  ...  1.6552  30.2453  30.2526  30.2588 -0.0184   
2391  999.88    0.0   0.02  ...  1.6709  30.2525  30.2594  30.2658 -0.0184   

       IMon7   IMon8            Comment       hv_grid        em  
2388 -0.0269 -0.0195  emission on again  2.347000e-07  0.000002  
2389 -0.0269 -0.0195  emission on again  2.357000e-07  0.000002  
2390 -0.02

count    1.810000e+02
mean     2.405948e-09
std      4.046104e-11
min      2.313591e-09
25%      2.376276e-09
50%      2.409788e-09
75%      2.440900e-09
max      2.473506e-09
Name: mass 2, corr, dtype: float64


C:\Users\etiirine\AppData\Local\Temp\ipykernel_19260\3449488278.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan


count    3.700000e+01
mean     3.827393e-09
std      1.471812e-09
min      2.904868e-09
25%      3.077445e-09
50%      3.355760e-09
75%      3.778370e-09
max      9.938567e-09
Name: mass 2, corr, dtype: float64

## Plotting

In [7]:
#Plotting the all the MID channels
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(40, 1.3e-10, '\u2190 the desorption peak of initial electron activity on the EGA', fontdict = font)
plt.text(2150, 7.35e-11, '\u2190 EGA turned', fontdict = font)
plt.text(600, 2e-11, 'No visible desorption during grid transparent-repelling cycles', fontdict = font)

plt.yscale('log')

In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(130, 1.9e-8, 'Pre-injection', fontdict = font)
plt.text(1400, 1.9e-6, 'First emission \u2192', fontdict = font)
plt.text(2770, 2.2e-6, 'Grid transparent', fontdict = font)
plt.text(3770, 9.5e-9, 'Grid repelling, EGA turned', fontdict = font)
plt.text(3525, 1.8e-6, 'Grid transparent', fontdict = font)

plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution @4K EGA on a-C with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(928, 1.3e-7, '\u2190 Injection complete', fontdict = font)
plt.text(1100, 1.6e-8, 'LHe refill', fontdict = font)
plt.text(2100, 4e-9, 'First emission', fontdict = font)
plt.text(4100, 2.4e-8, 'EGA rotated', fontdict = font)

ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()